In [10]:
import zstandard as zstd
import json
import os
from datetime import datetime
import logging.handlers
import re
import multiprocessing
import lzma

log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())

In [11]:
def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
    chunk = reader.read(chunk_size)
    bytes_read += chunk_size
    if previous_chunk is not None:
        chunk = previous_chunk + chunk
    try:
        return chunk.decode()
    except UnicodeDecodeError:
        if bytes_read > max_window_size:
            raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
        print(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
        return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_xz(file_name):
    with lzma.open(file_name, 'rb') as file_handle:
        buffer = ''
        while True:
            chunk = file_handle.read(2 ** 27)  # Change chunk size if necessary
            if not chunk:
                break
            
            chunk = chunk.decode()
            lines = (buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line, file_handle.tell()

            buffer = lines[-1]
            

def read_lines_zst(file_name):
    with open(file_name, 'rb') as file_handle:
        buffer = ''
        reader = zstd.ZstdDecompressor(max_window_size=2 ** 31).stream_reader(file_handle)
        while True:
            chunk = read_and_decode(reader, 2 ** 27, (2 ** 29) * 2)

            if not chunk:
                break
            lines = (buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line, file_handle.tell()

            buffer = lines[-1]

        reader.close()

In [12]:
def data_filter(input_data, file_extension = 'xz'):
    
    file_root_rc = '/shared/2/datasets/reddit-dump-all/RS/'
    output_root = '/shared/3/projects/multiparty-rivalry/data/Soccer/RS_raw/'

    field = "subreddit"
    
    # file paths setup
    file_path = file_root_rc + input_data + '.xz'
    output_path = output_root + 'filtered_' + input_data + '.json'
    
    # parameters for the loop record
    file_size = os.stat(file_path).st_size
    file_lines = 0
    file_bytes_processed = 0
    created = None
    bad_lines = 0

    # open files
    with open(output_path, 'w') as file:
        if file_extension == 'zst':
            reader = read_lines_zst(file_path)
        elif file_extension == 'xz':
            reader = read_lines_xz(file_path)
        else:
            raise ValueError("Unsupported file extension")
            
        for line, file_bytes_processed in reader:
            try:
                obj = json.loads(line)
                created = datetime.utcfromtimestamp(int(obj['created_utc']))
                
                if obj[field].lower() in subreddit_list:
                    
                    # print(json.dumps(obj, indent=4))  # show the formatted JSON data
                    obj_new = dict()
                    obj_new = obj
                    
#                     obj_new = dict()
#                     obj_new['author'] = obj['author']
#                     obj_new['author_fullname'] = obj['author_fullname']
#                     obj_new['author_created_utc'] = obj['author_created_utc']
#                     obj_new["author_flair_richtext"] = obj['author_flair_richtext']
#                     obj_new["author_flair_text"] = obj['author_flair_text']
                    
#                     obj_new['body'] = obj['body']
#                     obj_new['selftext'] = obj["selftext"]
#                     obj_new['controversiality'] = obj['controversiality']
#                     obj_new['created_utc'] = obj['created_utc']
#                     obj_new['id'] = obj['id']
#                     obj_new['is_submitter'] = obj['is_submitter']
#                     obj_new['link_id'] = obj['link_id']
#                     obj_new['name'] = obj['name']
#                     obj_new['parent_id'] = obj['parent_id']
#                     obj_new['permalink'] = obj['permalink']
#                     obj_new['score'] = obj['score']
#                     obj_new['subreddit'] = obj['subreddit']
#                     obj_new['subreddit_id'] = obj['subreddit_id']
#                     obj_new['subreddit_name_prefixed'] = obj['subreddit_name_prefixed']
#                     obj_new['subreddit_type'] = obj['subreddit_type']
#                     obj_new['total_awards_received'] = obj['total_awards_received']
   
#                     obj_new['num_comments'] = obj['num_comments']
#                     obj_new['num_crossposts'] = obj['num_crossposts']
#                     obj_new['retrieved_on'] = obj['retrieved_on']
#                     obj_new['retrieved_utc'] = obj['retrieved_utc']
#                     obj_new['selftext'] = obj['selftext']
#                     obj_new['title'] = obj['title']
#                     obj_new['upvote_ratio'] = obj['upvote_ratio']
#                     obj_new['view_count'] = obj['view_count']
#                     obj_new['wls'] = obj['wls']
                    # print(json.dumps(obj_new, indent=4))  # show the formatted JSON data
                    file.write(json.dumps(obj_new))
                    file.write('\n')
                    
            except (KeyError, json.JSONDecodeError) as err:
                bad_lines += 1
            file_lines += 1

            if file_lines % 100000 == 0:
                log.info(
                    f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")
    file.close()

In [13]:
import re

related_subreddit_path = '/shared/3/projects/multiparty-rivalry/data/Soccer/related_subreddits.txt'

with open(related_subreddit_path, "r") as file:
    lines = file.readlines()

subreddit_list = []
pattern = r"/r/(\w+)"


for line in lines:
    line = line.strip()
    subreddit_names = re.findall(pattern, line)
    subreddit_list += subreddit_names

subreddit_list.append('soccer')
subreddit_list = [subreddit.lower() for subreddit in subreddit_list]

print(subreddit_list[:5])

['bootroom', 'soccernoobs', 'madtekkers', 'supporters', 'futbol']


In [15]:
file_date = ['RS_2017-05', 'RS_2017-06', 'RS_2017-07', 'RS_2017-08', 'RS_2017-09', 'RS_2017-10', 'RS_2017-11', 'RS_2017-12',
            'RS_2018-01', 'RS_2018-02', 'RS_2018-03', 'RS_2018-04']

num_processes = 12
pool = multiprocessing.Pool(processes=num_processes)

# Use the Pool.map() method to distribute data and execute the function in parallel
pool.map(data_filter, file_date)

pool.close()
# pool.join()

2017-07-01 09:03:01 : 100,000 : 48 : 268,435,456:11%
2017-07-01 09:03:01 : 100,000 : 48 : 268,435,456:11%
2017-08-01 07:48:43 : 100,000 : 75 : 268,435,456:10%
2017-08-01 07:48:43 : 100,000 : 75 : 268,435,456:10%
2017-10-01 09:30:15 : 100,000 : 53 : 268,435,456:10%
2017-10-01 09:30:15 : 100,000 : 53 : 268,435,456:10%
2018-03-01 06:10:10 : 100,000 : 0 : 268,435,456:9%
2018-03-01 06:10:10 : 100,000 : 0 : 268,435,456:9%
2018-01-01 08:36:07 : 100,000 : 0 : 268,435,456:10%
2018-01-01 08:36:07 : 100,000 : 0 : 268,435,456:10%
2017-06-01 07:56:23 : 100,000 : 65 : 268,435,456:11%
2017-06-01 07:56:23 : 100,000 : 65 : 268,435,456:11%
2017-05-01 09:53:27 : 100,000 : 63 : 268,435,456:12%
2017-05-01 09:53:27 : 100,000 : 63 : 268,435,456:12%
2017-11-01 07:53:59 : 100,000 : 75 : 268,435,456:10%
2017-11-01 07:53:59 : 100,000 : 75 : 268,435,456:10%
2018-02-01 06:14:03 : 100,000 : 0 : 268,435,456:10%
2018-02-01 06:14:03 : 100,000 : 0 : 268,435,456:10%
2017-12-01 06:42:48 : 100,000 : 122 : 268,435,456:10%


2017-12-03 01:00:57 : 700,000 : 625 : 1,610,612,736:58%
2017-12-03 01:00:57 : 700,000 : 625 : 1,610,612,736:58%
2017-10-03 05:31:04 : 700,000 : 859 : 1,610,612,736:62%
2017-10-03 05:31:04 : 700,000 : 859 : 1,610,612,736:62%
2018-01-03 03:03:10 : 700,000 : 0 : 1,610,612,736:59%
2018-01-03 03:03:10 : 700,000 : 0 : 1,610,612,736:59%
2018-03-02 19:17:19 : 700,000 : 0 : 1,744,830,464:59%
2018-03-02 19:17:19 : 700,000 : 0 : 1,744,830,464:59%
2017-07-03 18:56:13 : 800,000 : 502 : 1,744,830,464:69%
2017-07-03 18:56:13 : 800,000 : 502 : 1,744,830,464:69%
2017-11-02 23:30:12 : 700,000 : 1,219 : 1,610,612,736:61%
2017-11-02 23:30:12 : 700,000 : 1,219 : 1,610,612,736:61%
2017-05-03 15:32:31 : 800,000 : 769 : 1,610,612,736:69%
2017-05-03 15:32:31 : 800,000 : 769 : 1,610,612,736:69%
2018-02-02 20:32:53 : 700,000 : 0 : 1,610,612,736:63%
2018-02-02 20:32:53 : 700,000 : 0 : 1,610,612,736:63%
2017-09-03 15:34:09 : 800,000 : 872 : 1,744,830,464:70%
2017-09-03 15:34:09 : 800,000 : 872 : 1,744,830,464:70%


2017-12-04 20:14:25 : 1,300,000 : 1,125 : 2,952,790,016:107%
2017-12-04 20:14:25 : 1,300,000 : 1,125 : 2,952,790,016:107%
2017-05-05 13:25:07 : 1,400,000 : 1,409 : 2,818,572,288:121%
2017-05-05 13:25:07 : 1,400,000 : 1,409 : 2,818,572,288:121%
2017-08-04 16:46:23 : 1,300,000 : 1,524 : 2,818,572,288:107%
2017-08-04 16:46:23 : 1,300,000 : 1,524 : 2,818,572,288:107%
2017-10-04 23:43:19 : 1,300,000 : 2,014 : 2,818,572,288:108%
2017-10-04 23:43:19 : 1,300,000 : 2,014 : 2,818,572,288:108%
2017-11-04 20:01:53 : 1,300,000 : 1,941 : 2,818,572,288:107%
2017-11-04 20:01:53 : 1,300,000 : 1,941 : 2,818,572,288:107%
2017-09-05 11:59:16 : 1,400,000 : 1,374 : 2,952,790,016:119%
2017-09-05 11:59:16 : 1,400,000 : 1,374 : 2,952,790,016:119%
2018-01-04 19:09:57 : 1,300,000 : 0 : 2,952,790,016:109%
2018-01-04 19:09:57 : 1,300,000 : 0 : 2,952,790,016:109%
2018-04-03 07:54:50 : 1,100,000 : 0 : 3,623,878,656:115%
2018-04-03 07:54:50 : 1,100,000 : 0 : 3,623,878,656:115%
2017-06-05 15:13:44 : 1,400,000 : 1,056 

2017-07-07 11:00:18 : 2,000,000 : 1,502 : 4,160,749,568:164%
2018-04-04 00:01:07 : 1,500,000 : 0 : 4,966,055,936:158%
2018-04-04 00:01:07 : 1,500,000 : 0 : 4,966,055,936:158%
2017-08-06 16:14:04 : 1,900,000 : 2,087 : 4,026,531,840:153%
2017-08-06 16:14:04 : 1,900,000 : 2,087 : 4,026,531,840:153%
2017-12-06 11:22:27 : 1,900,000 : 1,926 : 4,294,967,296:155%
2017-12-06 11:22:27 : 1,900,000 : 1,926 : 4,294,967,296:155%
2018-03-06 02:19:21 : 1,900,000 : 0 : 4,563,402,752:155%
2018-03-06 02:19:21 : 1,900,000 : 0 : 4,563,402,752:155%
2017-09-07 04:21:57 : 2,000,000 : 2,382 : 4,294,967,296:173%
2017-09-07 04:21:57 : 2,000,000 : 2,382 : 4,294,967,296:173%
2018-02-05 23:35:49 : 1,800,000 : 0 : 4,294,967,296:167%
2018-02-05 23:35:49 : 1,800,000 : 0 : 4,294,967,296:167%
2017-05-07 23:56:15 : 2,100,000 : 1,997 : 4,160,749,568:178%
2017-05-07 23:56:15 : 2,100,000 : 1,997 : 4,160,749,568:178%
2017-06-07 09:02:05 : 2,000,000 : 1,559 : 4,160,749,568:174%
2017-06-07 09:02:05 : 2,000,000 : 1,559 : 4,160,

2018-01-07 22:33:32 : 2,400,000 : 0 : 5,368,709,120:198%
2017-11-08 01:50:05 : 2,400,000 : 3,178 : 5,234,491,392:198%
2017-11-08 01:50:05 : 2,400,000 : 3,178 : 5,234,491,392:198%
2017-07-09 08:25:46 : 2,600,000 : 1,905 : 5,502,926,848:218%
2017-07-09 08:25:46 : 2,600,000 : 1,905 : 5,502,926,848:218%
2018-03-07 15:37:15 : 2,500,000 : 0 : 5,905,580,032:200%
2018-03-07 15:37:15 : 2,500,000 : 0 : 5,905,580,032:200%
2017-08-08 10:49:33 : 2,500,000 : 2,836 : 5,234,491,392:199%
2017-08-08 10:49:33 : 2,500,000 : 2,836 : 5,234,491,392:199%
2017-05-09 21:16:49 : 2,700,000 : 2,635 : 5,368,709,120:230%
2017-05-09 21:16:49 : 2,700,000 : 2,635 : 5,368,709,120:230%
2018-04-05 01:00:19 : 2,000,000 : 0 : 6,576,668,672:210%
2018-04-05 01:00:19 : 2,000,000 : 0 : 6,576,668,672:210%
2017-09-08 23:23:25 : 2,600,000 : 3,450 : 5,637,144,576:227%
2017-09-08 23:23:25 : 2,600,000 : 3,450 : 5,637,144,576:227%
2017-12-08 00:51:21 : 2,500,000 : 2,632 : 5,637,144,576:204%
2017-12-08 00:51:21 : 2,500,000 : 2,632 : 5,

...

2018-01-22 23:06:54 : 7,900,000 : 0 : 17,582,522,368:649%
2017-05-30 14:50:42 : 9,000,000 : 8,627 : 17,850,957,824:765%
2017-05-30 14:50:42 : 9,000,000 : 8,627 : 17,850,957,824:765%
2017-11-24 04:04:35 : 8,000,000 : 10,404 : 17,314,086,912:656%
2017-11-24 04:04:35 : 8,000,000 : 10,404 : 17,314,086,912:656%
2017-10-25 13:06:24 : 8,100,000 : 11,626 : 17,314,086,912:663%
2017-10-25 13:06:24 : 8,100,000 : 11,626 : 17,314,086,912:663%
2017-09-27 09:33:00 : 8,600,000 : 11,661 : 18,387,828,736:739%
2017-09-27 09:33:00 : 8,600,000 : 11,661 : 18,387,828,736:739%
2017-07-28 05:20:12 : 8,800,000 : 8,157 : 18,253,611,008:721%
2017-07-28 05:20:12 : 8,800,000 : 8,157 : 18,253,611,008:721%
2018-03-22 00:07:56 : 8,100,000 : 0 : 19,058,917,376:646%
2018-03-22 00:07:56 : 8,100,000 : 0 : 19,058,917,376:646%
2018-02-21 19:40:45 : 7,700,000 : 0 : 17,985,175,552:700%
2018-02-21 19:40:45 : 7,700,000 : 0 : 17,985,175,552:700%
2017-08-25 15:28:49 : 8,300,000 : 10,618 : 17,314,086,912:658%
2017-08-25 15:28:49 :

2018-02-22 23:59:29 : 8,200,000 : 0 : 19,193,135,104:747%
2018-02-22 23:59:29 : 8,200,000 : 0 : 19,193,135,104:747%
2017-07-30 03:23:39 : 9,400,000 : 8,578 : 19,595,788,288:775%
2017-07-30 03:23:39 : 9,400,000 : 8,578 : 19,595,788,288:775%
2018-01-24 14:17:03 : 8,500,000 : 0 : 18,924,699,648:698%
2018-01-24 14:17:03 : 8,500,000 : 0 : 18,924,699,648:698%
2017-12-27 05:47:49 : 9,000,000 : 8,450 : 20,266,876,928:732%
2017-12-27 05:47:49 : 9,000,000 : 8,450 : 20,266,876,928:732%
2018-03-23 13:34:07 : 8,700,000 : 0 : 20,535,312,384:696%
2018-03-23 13:34:07 : 8,700,000 : 0 : 20,535,312,384:696%
2017-06-29 05:43:51 : 9,000,000 : 8,082 : 18,522,046,464:773%
2017-06-29 05:43:51 : 9,000,000 : 8,082 : 18,522,046,464:773%
2017-11-26 01:45:29 : 8,600,000 : 11,068 : 18,656,264,192:707%
2017-11-26 01:45:29 : 8,600,000 : 11,068 : 18,656,264,192:707%
2017-08-27 15:29:20 : 8,900,000 : 11,307 : 18,522,046,464:704%
2017-08-27 15:29:20 : 8,900,000 : 11,307 : 18,522,046,464:704%
2017-10-27 03:57:25 : 8,700,

2018-01-26 00:32:46 : 9,100,000 : 0 : 20,266,876,928:748%
2018-03-25 02:39:10 : 9,300,000 : 0 : 21,877,489,664:741%
2018-03-25 02:39:10 : 9,300,000 : 0 : 21,877,489,664:741%
2017-08-29 09:53:08 : 9,500,000 : 12,052 : 19,730,006,016:750%
2017-08-29 09:53:08 : 9,500,000 : 12,052 : 19,730,006,016:750%
2017-11-27 21:01:42 : 9,200,000 : 11,704 : 19,998,441,472:758%
2017-11-27 21:01:42 : 9,200,000 : 11,704 : 19,998,441,472:758%
2017-10-29 02:37:38 : 9,300,000 : 13,380 : 19,998,441,472:765%
2017-10-29 02:37:38 : 9,300,000 : 13,380 : 19,998,441,472:765%
2017-12-29 06:35:03 : 9,700,000 : 9,070 : 21,877,489,664:791%
2017-12-29 06:35:03 : 9,700,000 : 9,070 : 21,877,489,664:791%
2018-02-24 20:45:03 : 8,900,000 : 0 : 20,803,747,840:809%
2018-02-24 20:45:03 : 8,900,000 : 0 : 20,803,747,840:809%
2018-03-25 12:05:07 : 9,400,000 : 0 : 22,145,925,120:750%
2018-03-25 12:05:07 : 9,400,000 : 0 : 22,145,925,120:750%
2018-01-26 07:26:11 : 9,200,000 : 0 : 20,535,312,384:758%
2018-01-26 07:26:11 : 9,200,000 : 

2018-03-27 12:56:29 : 10,200,000 : 0 : 24,024,973,312:814%
2018-03-27 12:56:29 : 10,200,000 : 0 : 24,024,973,312:814%
2017-12-31 19:29:58 : 10,500,000 : 9,587 : 23,756,537,856:858%
2017-12-31 19:29:58 : 10,500,000 : 9,587 : 23,756,537,856:858%
2017-11-30 05:45:48 : 10,100,000 : 12,902 : 21,877,489,664:829%
2017-11-30 05:45:48 : 10,100,000 : 12,902 : 21,877,489,664:829%
2018-02-26 22:59:53 : 9,700,000 : 0 : 22,682,796,032:882%
2018-02-26 22:59:53 : 9,700,000 : 0 : 22,682,796,032:882%
2017-10-31 19:26:10 : 10,200,000 : 15,050 : 21,877,489,664:837%
2017-10-31 19:26:10 : 10,200,000 : 15,050 : 21,877,489,664:837%
2018-01-28 16:19:11 : 10,000,000 : 0 : 22,414,360,576:827%
2018-01-28 16:19:11 : 10,000,000 : 0 : 22,414,360,576:827%
2018-04-20 02:46:16 : 7,900,000 : 0 : 25,501,368,320:813%
2018-04-20 02:46:16 : 7,900,000 : 0 : 25,501,368,320:813%
2018-03-27 17:42:26 : 10,300,000 : 0 : 24,293,408,768:823%
2018-03-27 17:42:26 : 10,300,000 : 0 : 24,293,408,768:823%
2017-11-30 14:20:59 : 10,200,000

2018-04-25 22:32:37 : 10,100,000 : 0 : 32,614,907,904:1039%
2018-04-26 04:28:12 : 10,200,000 : 0 : 32,883,343,360:1048%
2018-04-26 04:28:12 : 10,200,000 : 0 : 32,883,343,360:1048%
2018-04-26 12:26:10 : 10,300,000 : 0 : 33,285,996,544:1061%
2018-04-26 12:26:10 : 10,300,000 : 0 : 33,285,996,544:1061%
2018-04-26 17:27:22 : 10,400,000 : 0 : 33,554,432,000:1069%
2018-04-26 17:27:22 : 10,400,000 : 0 : 33,554,432,000:1069%
2018-04-26 22:18:39 : 10,500,000 : 0 : 33,822,867,456:1078%
2018-04-26 22:18:39 : 10,500,000 : 0 : 33,822,867,456:1078%
2018-04-27 04:19:56 : 10,600,000 : 0 : 34,225,520,640:1091%
2018-04-27 04:19:56 : 10,600,000 : 0 : 34,225,520,640:1091%
2018-04-27 12:15:02 : 10,700,000 : 0 : 34,493,956,096:1099%
2018-04-27 12:15:02 : 10,700,000 : 0 : 34,493,956,096:1099%
2018-04-27 17:23:35 : 10,800,000 : 0 : 34,896,609,280:1112%
2018-04-27 17:23:35 : 10,800,000 : 0 : 34,896,609,280:1112%
2018-04-27 22:38:06 : 10,900,000 : 0 : 35,165,044,736:1120%
2018-04-27 22:38:06 : 10,900,000 : 0 : 3